In [5]:
%display latex
%runfile character.py

On note $\Delta$ un déterminant de type Vandermonde.

``v = vandermonde(Diagram)``

In [3]:
H = DerivativeVandermondeSpaceWithInert(QQ, Diagram([(0,0),(2,0),(4,0)]))
v = H.vandermonde()
v

-x00^4*x01^2 + x00^2*x01^4 + x00^4*x02^2 - x01^4*x02^2 - x00^2*x02^4 + x01^2*x02^4

On constate que $p_1(\partial)\Delta$

In [4]:
x = H._P.variables()[0]
sum(v.derivative(x[i]) for i in range(0,3))

-2*x00^4*x01 - 4*x00^3*x01^2 + 4*x00^2*x01^3 + 2*x00*x01^4 + 2*x00^4*x02 - 2*x01^4*x02 + 4*x00^3*x02^2 - 4*x01^3*x02^2 - 4*x00^2*x02^3 + 4*x01^2*x02^3 - 2*x00*x02^4 + 2*x01*x02^4

est différent de $\sum_i y_i \partial_{x_i}^2 \sum_j x_j \partial_{y_j} \Delta$.

In [5]:
P = DiagonalPolynomialRing(QQ, 3, 2)
x = P.variables()[0]
y = P.variables()[1]
v = P(v)
sum(x[i]*sum(y[j]*v.derivative(x[j],2) for j in range(0,3)).derivative(y[i]) for i in range(0,3))

-2*x00^4*x01 - 12*x00^3*x01^2 + 12*x00^2*x01^3 + 2*x00*x01^4 + 2*x00^4*x02 - 2*x01^4*x02 + 12*x00^3*x02^2 - 12*x01^3*x02^2 - 12*x00^2*x02^3 + 12*x01^2*x02^3 - 2*x00*x02^4 + 2*x01*x02^4

Or le commutateur de $\sum_i y_i \partial_{x_i}^2$ et $\sum_j x_j \partial_{y_j}$ nous donne $\sum_i x_i \partial_{x_i}^2$. On va donc essayer d'ajouter cet opérateur dans nos calculs.

### Sans quotient

Pour vérifier que tout est correct au niveau de la polarisation en un jeu de variables, on prend tous les opérateurs ensemble (dérivées partielles, polarisateurs et $\sum_i x_i \partial_{x_i}^k$ pour k=2,3) et on génère le sous-espace.

``V = Isotyp(Subspace(Delta, list_op))
charac(V)``

In [6]:
H = DerivativeVandermondeSpaceWithInert(QQ, Diagram([(0,0),(2,0),(4,0)]))
P = DiagonalPolynomialRing(QQ, 3, 2)
v = P(H.vandermonde())
d=4
n=3

X = P.variables()[0]
Y = P.variables()[1]
D = P._grading_set

s = SymmetricFunctions(QQ).s()

operators = {}

def make_pol_comp_young(X, Y, k, mu):
    def f(p):
        return apply_young_idempotent(sum(Y[i]*p.derivative(X[i],k) for i in range(0,len(X))), mu)
    return f

for nu in Partitions(3):
    operators[(D((-1,0)),nu)] = [make_deriv_comp_young(X[i], nu) for i in range(0,n)]
    operators[(D((0,-1)),nu)] = [make_deriv_comp_young(Y[i], nu) for i in range(0,n)]
    
    operators[(D((-1,0)),nu)] += [make_deriv_comp_young2(X, 2, nu)]
    operators[(D((-2,0)),nu)] = [make_deriv_comp_young2(X, 3, nu)]
    
    for k in range(1,d+1):
        operators[(D((-k,1)),nu)] = [make_pol_comp_young(X, Y, k, nu)]
        operators[(D((1,-k)),nu)] = [make_pol_comp_young(Y, X, k, nu)]
        
generators={(D((6,0)),Partition([1,1,1])):[v]}

basis = Subspace(generators=generators, operators=operators, add_degrees=add_degree_isotyp).basis()

for nu in Partitions(3):
    charac = 0
    print nu
    for key in basis.iterkeys():
        if nu == key[1]:
            charac += len(basis[key])*P.multipower(key[0])
    print charac
        
    charac = (s.from_polynomial(charac)).restrict_partition_lengths(2,exact=False)
    print charac
    print

[3]
q0^3 + q0^2*q1 + q0*q1^2 + q1^3 + 2*q0^2 + 2*q0*q1 + 2*q1^2 + q0 + q1 + 1
s[] + s[1] + 2*s[2] + s[3]

[2, 1]
q0^5 + q0^4*q1 + q0^3*q1^2 + q0^2*q1^3 + q0*q1^4 + q1^5 + 3*q0^4 + 4*q0^3*q1 + 4*q0^2*q1^2 + 4*q0*q1^3 + 3*q1^4 + 3*q0^3 + 5*q0^2*q1 + 5*q0*q1^2 + 3*q1^3 + 2*q0^2 + 3*q0*q1 + 2*q1^2 + q0 + q1
s[1] + s[1, 1] + 2*s[2] + 2*s[2, 1] + 3*s[3] + s[3, 1] + 3*s[4] + s[5]

[1, 1, 1]
q0^6 + q0^5*q1 + q0^4*q1^2 + q0^3*q1^3 + q0^2*q1^4 + q0*q1^5 + q1^6 + 2*q0^5 + 3*q0^4*q1 + 3*q0^3*q1^2 + 3*q0^2*q1^3 + 3*q0*q1^4 + 2*q1^5 + q0^4 + 3*q0^3*q1 + 3*q0^2*q1^2 + 3*q0*q1^3 + q1^4 + q0^3 + 2*q0^2*q1 + 2*q0*q1^2 + q1^3 + q0*q1
s[1, 1] + s[2, 1] + s[3] + 2*s[3, 1] + s[4] + s[4, 1] + 2*s[5] + s[6]



Si on restreint les résultats précédents à un jeu de variables on obtient bien la même chose qu'avec le calcul optimisé. 

``V = Isotyp(Subspace(vandermonde(Diagram),list_op),mu)``

In [7]:
character_by_isotypic_plain(Diagram([(0,0),(2,0),(4,0)]), Partition([3]), r=1, quotient=False)
character_by_isotypic_plain(Diagram([(0,0),(2,0),(4,0)]), Partition([2,1]), r=1, quotient=False)
character_by_isotypic_plain(Diagram([(0,0),(2,0),(4,0)]), Partition([1,1,1]), r=1, quotient=False)

[3]
charac :  s[] + s[1] + 2*s[2] + s[3]

[2, 1]
charac :  s[1] + 2*s[2] + 3*s[3] + 3*s[4] + s[5]

[1, 1, 1]
charac :  s[3] + s[4] + 2*s[5] + s[6]



{(3,): 1, (4,): 1, (5,): 2, (6,): 1}

### Avec quotient

On essaye maintenant d'ajouter les $\sum_i x_i \partial_{x_i}^k$ dans le quotient. On commence par regarder si ça fonctionne pour un seul jeu de variables. 

``V1 = Isotyp(Subspace(Delta, list_derivatives), mu)
V2 = Quotient(V1, list_op)
charac(V1)-charac(V2)``

In [6]:
character_by_isotypic_plain(Diagram([(0,0),(2,0),(4,0)]), Partition([1,1,1]), r=1, quotient=True, use_steenrod_op=False)
character_by_isotypic_plain(Diagram([(0,0),(2,0),(4,0)]), Partition([2,1]), r=1, quotient=True, use_steenrod_op=False)
character_by_isotypic_plain(Diagram([(0,0),(2,0),(4,0)]), Partition([3]), r=1, quotient=True, use_steenrod_op=False)

[1, 1, 1]
quotient : 
s[3] + s[4] + s[5]

avant quotient : 
s[3] + s[4] + s[5] + s[6]

charac :  s[6]

[2, 1]
quotient : 
s[1] + 2*s[2] + 2*s[3] + s[4]

avant quotient : 
s[1] + 2*s[2] + 2*s[3] + 2*s[4] + s[5]

charac :  s[4] + s[5]

[3]
quotient : 
s[] + s[1] + s[2]

avant quotient : 
s[] + s[1] + s[2] + s[3]

charac :  s[3]



{(3,): 1}

In [43]:
character_by_isotypic_plain(Diagram([(0,0),(1,0),(3,0)]), Partition([1,1,1]), r=1, quotient=True, use_steenrod_op=False)
character_by_isotypic_plain(Diagram([(0,0),(1,0),(3,0)]), Partition([2,1]), r=1, quotient=True, use_steenrod_op=False)
character_by_isotypic_plain(Diagram([(0,0),(1,0),(3,0)]), Partition([3]), r=1, quotient=True, use_steenrod_op=False)

[1, 1, 1]
quotient : 
s[3]

avant quotient : 
s[3] + s[4]

charac :  s[4]

[2, 1]
quotient : 
s[1] + s[2]

avant quotient : 
s[1] + 2*s[2] + s[3]

charac :  s[2] + s[3]

[3]
quotient : 
s[]

avant quotient : 
s[] + s[1]

charac :  s[1]



{(1,): 1}

``V1 = Isotyp(Subspace(Delta, list_derivatives_steenrod), mu)
V2 = Quotient(V1, list_op)
charac(V1)-charac(V2)``

In [44]:
character_by_isotypic_plain(Diagram([(0,0),(2,0),(4,0)]), Partition([1,1,1]), r=1, quotient=True, use_steenrod_op=True)
character_by_isotypic_plain(Diagram([(0,0),(2,0),(4,0)]), Partition([2,1]), r=1, quotient=True, use_steenrod_op=True)
character_by_isotypic_plain(Diagram([(0,0),(2,0),(4,0)]), Partition([3]), r=1, quotient=True, use_steenrod_op=True)

[1, 1, 1]
quotient : 
s[3] + s[4] + 2*s[5]

avant quotient : 
s[3] + s[4] + 2*s[5] + s[6]

charac :  s[6]

[2, 1]
quotient : 
s[1] + 2*s[2] + 3*s[3] + 2*s[4]

avant quotient : 
s[1] + 2*s[2] + 3*s[3] + 3*s[4] + s[5]

charac :  s[4] + s[5]

[3]
quotient : 
s[] + s[1] + 2*s[2]

avant quotient : 
s[] + s[1] + 2*s[2] + s[3]

charac :  s[3]



{(3,): 1}

In [45]:
character_by_isotypic_plain(Diagram([(0,0),(1,0),(3,0)]), Partition([1,1,1]), r=1, quotient=True, use_steenrod_op=True)
character_by_isotypic_plain(Diagram([(0,0),(1,0),(3,0)]), Partition([2,1]), r=1, quotient=True, use_steenrod_op=True)
character_by_isotypic_plain(Diagram([(0,0),(1,0),(3,0)]), Partition([3]), r=1, quotient=True, use_steenrod_op=True)

[1, 1, 1]
quotient : 
s[3]

avant quotient : 
s[3] + s[4]

charac :  s[4]

[2, 1]
quotient : 
s[1] + 2*s[2]

avant quotient : 
s[1] + 2*s[2] + s[3]

charac :  s[3]

[3]
quotient : 
s[]

avant quotient : 
s[] + s[1]

charac :  s[1]



{(1,): 1}

Puis essais avec polarisation

``V1 = Subspace(Isotyp(Subspace(Delta, list_derivatives_steenrod), mu), list_polariz)
V2 = Quotient(V1, list_op)
charac(V1)-charac(V2)``

In [33]:
v = DerivativeVandermondeSpaceWithInert(QQ, Diagram([(0,0),(1,0),(3,0)])).vandermonde()
v

-x00^3*x01 + x00*x01^3 + x00^3*x02 - x01^3*x02 - x00*x02^3 + x01*x02^3

In [34]:
P = DiagonalPolynomialRing(QQ, 3, 2)
x = P.variables()[0]
y = P.variables()[1]
v = P(v)

In [35]:
sum(v.derivative(x[i]) for i in range(0,3))

-3*x00^2*x01 + 3*x00*x01^2 + 3*x00^2*x02 - 3*x01^2*x02 - 3*x00*x02^2 + 3*x01*x02^2

In [39]:
f1 = sum(y[i]*v.derivative(x[i],2) for i in range(0,3))
sum(x[i]*f1.derivative(y[i]) for i in range(0,3))

-6*x00^2*x01 + 6*x00*x01^2 + 6*x00^2*x02 - 6*x01^2*x02 - 6*x00*x02^2 + 6*x01*x02^2

In [40]:
sum(x[i]*v.derivative(x[i],2) for i in range(0,3))

-6*x00^2*x01 + 6*x00*x01^2 + 6*x00^2*x02 - 6*x01^2*x02 - 6*x00*x02^2 + 6*x01*x02^2

In [69]:
c1 = character_with_inert(Diagram([(0,0),(2,0),(4,0)]), quotient=False)
s = SymmetricFunctions(QQ).s()
c2 = s[1, 1].tensor(s[3]) + s[2, 1].tensor(s[2, 1]) + s[2, 2].tensor(s[1, 1, 1]) + s[3].tensor(s[3]) + s[3, 1].tensor( s[2, 1]) + s[4].tensor( s[2, 1]) + s[4, 1].tensor(s[1, 1, 1]) + s[5].tensor( s[2, 1]) + s[6].tensor(s[1, 1, 1])
c1-c2

[3]
charac :  s[] + s[1] + 2*s[2] + s[3]

[2, 1]
charac :  s[1] + 2*s[2] + 3*s[3] + 3*s[4] + s[5]

[1, 1, 1]
charac :  s[3] + s[4] + 2*s[5] + s[6]



s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] - s[1, 1] # s[3] + 2*s[2] # s[2, 1] + 2*s[2] # s[3] - s[2, 1] # s[2, 1] - s[2, 2] # s[1, 1, 1] + s[3] # s[1, 1, 1] + 3*s[3] # s[2, 1] - s[3, 1] # s[2, 1] + s[4] # s[1, 1, 1] + 2*s[4] # s[2, 1] - s[4, 1] # s[1, 1, 1] + 2*s[5] # s[1, 1, 1]

In [58]:
tensor([m,s])(c1-c2)

-m[1, 1, 1] # s[2, 1] - 2*m[1, 1, 1, 1] # s[1, 1, 1] - 2*m[1, 1, 1, 1] # s[2, 1] - 3*m[1, 1, 1, 1, 1] # s[1, 1, 1] + m[2] # s[3] - m[2, 1, 1] # s[1, 1, 1] - m[2, 1, 1] # s[2, 1] - 2*m[2, 1, 1, 1] # s[1, 1, 1] - m[2, 2] # s[1, 1, 1] - m[2, 2, 1] # s[1, 1, 1] + m[3] # s[2, 1] - m[3, 1, 1] # s[1, 1, 1] + m[4] # s[2, 1] + m[5] # s[1, 1, 1]

In [52]:
c1

s[2] # s[3] + s[3] # s[2, 1] + s[3] # s[3] + 2*s[4] # s[2, 1] + s[5] # s[1, 1, 1] + s[5] # s[2, 1] + s[6] # s[1, 1, 1]

In [49]:
c2

s[1, 1] # s[3] + s[2, 1] # s[2, 1] + s[2, 2] # s[1, 1, 1] + s[3] # s[3] + s[3, 1] # s[2, 1] + s[4] # s[2, 1] + s[4, 1] # s[1, 1, 1] + s[5] # s[2, 1] + s[6] # s[1, 1, 1]

In [70]:
def Crache(F,w=None,g=None):
    if F==0: 
        show()
    elif g==None:
        Dishout(F,s,s)
    else:
        n=3
        for mu in sorted(list(Partitions(n))):
            val=w(Scalar2(F,g(mu)))
            if val<>0:
                show((tuple(mu),val))

@cached_function
def Scalar2(U,f): 
    res=add(_c*s(_a)*(s(_b).scalar(f)) for (_a,_b),_c in tensor([s,s])(U))
    if res==None: 
        return 0*Un
    else:
        return res

In [71]:
Crache(c1-c2,m,s)

((1, 1, 1),
 m[1, 1, 1] - m[1, 1, 1, 1] - 2*m[1, 1, 1, 1, 1] + m[2, 1] - m[2, 1, 1, 1] + m[3] + m[3, 1] + m[3, 2] + m[4] + m[4, 1] + 2*m[5])

((2, 1),
 m[1] + 2*m[1, 1] + m[1, 1, 1] - m[1, 1, 1, 1] + 2*m[2] + 2*m[2, 1] + m[2, 2] + 3*m[3] + m[3, 1] + 2*m[4])

((3,), m[] + m[1] + m[1, 1] + 2*m[2])

In [79]:
v = DerivativeVandermondeSpaceWithInert(QQ, Diagram([(0,0),(2,0),(4,0)])).vandermonde()
v = P(v)
v

-x00^4*x01^2 + x00^2*x01^4 + x00^4*x02^2 - x01^4*x02^2 - x00^2*x02^4 + x01^2*x02^4

In [81]:
for i in range (0,3) :
    show(apply_young_idempotent(v.derivative(x[i]), Partition([1,1,1])))

-4*x00^4*x01 - 8*x00^3*x01^2 + 8*x00^2*x01^3 + 4*x00*x01^4 + 4*x00^4*x02 - 4*x01^4*x02 + 8*x00^3*x02^2 - 8*x01^3*x02^2 - 8*x00^2*x02^3 + 8*x01^2*x02^3 - 4*x00*x02^4 + 4*x01*x02^4

-4*x00^4*x01 - 8*x00^3*x01^2 + 8*x00^2*x01^3 + 4*x00*x01^4 + 4*x00^4*x02 - 4*x01^4*x02 + 8*x00^3*x02^2 - 8*x01^3*x02^2 - 8*x00^2*x02^3 + 8*x01^2*x02^3 - 4*x00*x02^4 + 4*x01*x02^4

-4*x00^4*x01 - 8*x00^3*x01^2 + 8*x00^2*x01^3 + 4*x00*x01^4 + 4*x00^4*x02 - 4*x01^4*x02 + 8*x00^3*x02^2 - 8*x01^3*x02^2 - 8*x00^2*x02^3 + 8*x01^2*x02^3 - 4*x00*x02^4 + 4*x01*x02^4